# Predicting COVID-19 cases in Ireland

## Import the Libraries

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

### Import the Datasets
- Data for COVID cases the Republic of Ireland was sourced from Our World in Data
- Data for the prevalence of COVID-19 internet searches were sourced from Google Trends
- Data for COVID cases in Northern Ireland was sourced from the Department of Health for Northern Ireland 

In [ ]:
# Import Republic of Ireland and Google Trends data
roi_data = pd.read_excel("preparedData.xlsx")

In [ ]:
# Import Northern Ireland data
#ni_data = pd.read_excel("ni_smoothed.xlsx")

In [ ]:
# Merge the datasets
#data = pd.merge(roi_data, ni_data, left_on='date', right_on='date', how='left')

In [ ]:
data = roi_data.copy(deep=True)

In [ ]:
data.fillna(0, inplace=True)

In [ ]:
#data['new_cases_smoothed'] = data['new_cases_smoothed']+data['newCasesByPublishDateSmoothed']

In [ ]:
data.head(5)

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data.sort_values(by='date', ascending=True, inplace=True)
data = data.set_index('date')

In [ ]:
data.drop(columns=['tests_per_case', 'new_cases','new_tests','reproduction_rate',
                   'positive_rate','total_vaccinations','stringency_index',
            'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations','population',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'life_expectancy', 'human_development_index', 
       'COVID-19 testing: (Ireland)', 'COVID-19 rapid antigen test: (Ireland)',
       'Health Service Executive: (Ireland)', 'Vaccination: (Ireland)',
       'book covid test: (Ireland)_x', 'how many covid cases today: (Ireland)',
       'pcr covid test: (Ireland)', 'close contact covid: (Ireland)',
       'book a covid test: (Ireland)', 'vaccination centre: (Ireland)',
       'pharmacy near me: (Ireland)',
       'Treatment and management of COVID-19: (Ireland)',
       'Hand sanitizer: (Ireland)', 'Face mask: (Ireland)',
       'book covid test: (Ireland)_y', 'covid test dublin: (Ireland)',
       'covid test centre: (Ireland)', 'hse covid vaccine: (Ireland)',
       'hse vaccine portal: (Ireland)', 'hse portal vaccine: (Ireland)',
       'pcr test hse: (Ireland)', 'hse covid test: (Ireland)',
       'hse vaccine registration: (Ireland)',
       'how long will it take to vaccinate ireland: (Ireland)'],inplace=True)

In [ ]:
data.describe().transpose()

In [ ]:
# Take a copy of dataframe without NaN values
df = data.dropna()

In [ ]:
df.tail()

In [ ]:
df.loc[:,'ncs'] = df.loc[:,'new_cases_smoothed'].shift(5)
df.loc[:,'ncs_diff'] = df.loc[:,'ncs'].diff()
df.loc[:,'ncs2'] = df.loc[:,'ncs'].shift()
df.loc[:,'ncs_diff2'] = df.loc[:,'ncs2'].diff()
df.loc[:,'ncs3'] = df.loc[:,'ncs2'].shift()
df.loc[:,'ncs_diff3'] = df.loc[:,'ncs3'].diff()
df.loc[:,'ncs4'] = df.loc[:,'ncs3'].shift()
df.loc[:,'ncs_diff4'] = df.loc[:,'ncs4'].diff()
df.loc[:,'ncs5'] = df.loc[:,'ncs4'].shift()
df.loc[:,'ncs_diff5'] = df.loc[:,'ncs5'].diff()
df.loc[:,'ncs6'] = df.loc[:,'ncs5'].shift()
df.loc[:,'ncs_diff6'] = df.loc[:,'ncs6'].diff()
df.loc[:,'ncs7'] = df.loc[:,'ncs6'].shift()
df.loc[:,'ncs_diff7'] = df.loc[:,'ncs7'].diff()

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

# USE THIS

df.loc[:,'google'] = df.loc[:,'covid: (Ireland)'].shift(5)
df.loc[:,'google_diff'] = df.loc[:,'google'].diff()
df.loc[:,'google2'] = df.loc[:,'google'].shift()
df.loc[:,'google_diff2'] = df.loc[:,'google2'].diff()
df.loc[:,'google3'] = df.loc[:,'google2'].shift()
df.loc[:,'google_diff3'] = df.loc[:,'google3'].diff()
df.loc[:,'google4'] = df.loc[:,'google3'].shift()
df.loc[:,'google_diff4'] = df.loc[:,'google4'].diff()
df.loc[:,'google5'] = df.loc[:,'google4'].shift()
df.loc[:,'google_diff5'] = df.loc[:,'google5'].diff()
df.loc[:,'google6'] = df.loc[:,'google5'].shift()
df.loc[:,'google_diff6'] = df.loc[:,'google6'].diff()
df.loc[:,'google7'] = df.loc[:,'google6'].shift()
df.loc[:,'google_diff7'] = df.loc[:,'google7'].diff()

In [ ]:
df = df.dropna()

In [ ]:
#df = df.fillna(method='ffill').fillna(method='bfill')

In [ ]:
df.head(5)

In [ ]:

df.tail(5)

In [ ]:
df.shape


In [ ]:
n = len(df)

In [ ]:
df.columns

In [ ]:
variables = df.columns.drop(['covid: (Ireland)', 'ncs', 'ncs_diff', 'ncs2',
       'ncs_diff2', 'ncs3', 'ncs_diff3', 'ncs4', 'ncs_diff4', 'ncs5',
       'ncs_diff5', 'ncs6', 'ncs_diff6', 'ncs7', 'ncs_diff7'])

In [ ]:
X_train = df.iloc[0:int(n*0.7)].drop(['new_cases_smoothed'], axis=1)
y_train = df.iloc[0:int(n*0.7)].drop(columns=variables, axis=1)

In [ ]:
X_test = df.iloc[int(n*0.7):int(n*0.9)].drop(['new_cases_smoothed'],axis=1)
y_test = df.iloc[int(n*0.7):int(n*0.9)].drop(columns=variables, axis=1)

In [ ]:
X_val = df.iloc[int(n*0.9):].drop(['new_cases_smoothed'], axis=1)
y_val = df.iloc[int(n*0.9):].drop(columns=variables, axis=1)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
X_val.head()

In [ ]:
y_val.head()

In [ ]:
model = MLPRegressor()

In [ ]:
param_search = {
    "hidden_layer_sizes": [(1,),(50,),(100,),(150,),(200,),(250,),(300,),(350,)],
    "activation": ["identity", "logistic", "tanh", "relu"],
    "solver": ["lbfgs", "sgd", "adam"],
    "alpha": [0.00005,0.0005, 0.005],
    "learning_rate": ['constant', 'invscaling', 'adaptive']
}

In [ ]:
tsvc = TimeSeriesSplit(n_splits=5)
gsearch = GridSearchCV(estimator=model, cv=tsvc, param_grid=param_search, scoring='r2')
gsearch.fit(X_train, y_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

In [ ]:
print(best_model)

In [ ]:
print(best_score)

In [ ]:
y_pred = best_model.predict(X_test)

print(r2_score(y_test, y_pred))

In [ ]:
y_val_pred = best_model.predict(X_val)

In [ ]:
print(r2_score(y_val, y_val_pred))

In [ ]:
y_test

In [ ]:
y_test['predictions'] = y_pred.tolist()


In [ ]:
y_test.reset_index(inplace=True)
y_test.plot(x='date', y=['new_cases_smoothed','predictions'], linestyle="dashed", figsize=(10,10))


In [ ]:

y_val['predictions'] = y_val_pred.tolist()

In [ ]:
y_val.reset_index(inplace=True)

In [ ]:
y_val.plot(x='date', y=['new_cases_smoothed', 'predictions'], linestyle="dashed", figsize=(10,10))

In [ ]:
data.tail(25)

In [ ]:
owid = pd.read_csv("owid-covid-data(1).csv")

In [ ]:
trend = pd.read_csv("multiTimeline(4).csv", skiprows=2)

In [ ]:
owid.tail()

In [ ]:
trend.head()

In [ ]:
combi = pd.merge(owid, trend, left_on='date', right_on='Day', how='left')

In [ ]:
combi['new_cases_smoothed'].fillna(method='ffill', inplace=True)

In [ ]:
combi.tail()

In [ ]:
combi.drop(columns=['Day'], inplace=True)

In [ ]:
combi.loc[:,'ncs'] = combi.loc[:,'new_cases_smoothed'].shift(5)
combi.loc[:,'ncs_diff'] = combi.loc[:,'ncs'].diff()
combi.loc[:,'ncs2'] = combi.loc[:,'ncs'].shift()
combi.loc[:,'ncs_diff2'] = combi.loc[:,'ncs2'].diff()
combi.loc[:,'ncs3'] = combi.loc[:,'ncs2'].shift()
combi.loc[:,'ncs_diff3'] = combi.loc[:,'ncs3'].diff()
combi.loc[:,'ncs4'] = combi.loc[:,'ncs3'].shift()
combi.loc[:,'ncs_diff4'] = combi.loc[:,'ncs4'].diff()
combi.loc[:,'ncs5'] = combi.loc[:,'ncs4'].shift()
combi.loc[:,'ncs_diff5'] = combi.loc[:,'ncs5'].diff()
combi.loc[:,'ncs6'] = combi.loc[:,'ncs5'].shift()
combi.loc[:,'ncs_diff6'] = combi.loc[:,'ncs6'].diff()
combi.loc[:,'ncs7'] = combi.loc[:,'ncs6'].shift()
combi.loc[:,'ncs_diff7'] = combi.loc[:,'ncs7'].diff()

In [ ]:
combi.loc[:,'google'] = combi.loc[:,'COVID: (Ireland)'].shift(5)
combi.loc[:,'google_diff'] = combi.loc[:,'google'].diff()
combi.loc[:,'google2'] = combi.loc[:,'google'].shift()
combi.loc[:,'google_diff2'] = combi.loc[:,'google2'].diff()
combi.loc[:,'google3'] = combi.loc[:,'google2'].shift()
combi.loc[:,'google_diff3'] = combi.loc[:,'google3'].diff()
combi.loc[:,'google4'] = combi.loc[:,'google3'].shift()
combi.loc[:,'google_diff4'] = combi.loc[:,'google4'].diff()
combi.loc[:,'google5'] = combi.loc[:,'google4'].shift()
combi.loc[:,'google_diff5'] = combi.loc[:,'google5'].diff()
combi.loc[:,'google6'] = combi.loc[:,'google5'].shift()
combi.loc[:,'google_diff6'] = combi.loc[:,'google6'].diff()
combi.loc[:,'google7'] = combi.loc[:,'google6'].shift()
combi.loc[:,'google_diff7'] = combi.loc[:,'google7'].diff()

In [ ]:
combi.head()

In [ ]:
combi.tail(30)

In [ ]:
combi['date'] = pd.to_datetime(combi['date'])

In [ ]:
dates = combi.iloc[729:743,0]

In [ ]:
x_combi_slice = combi.iloc[717:731,:]

In [ ]:
x_combi_slice.drop(columns=['new_cases_smoothed','COVID: (Ireland)'], inplace=True)

In [ ]:
x_combi_slice.set_index('date', inplace=True)

In [ ]:
x_combi_slice

In [ ]:
x_combi_slice.fillna(method='ffill', inplace=True)

In [ ]:
y_combi_slice = combi.iloc[729:743,1]

In [ ]:
y_combi_slice.shape

In [ ]:
y_combi_slice = y_combi_slice.to_frame()


In [ ]:
y_combi_slice_pred = best_model.predict(x_combi_slice)

In [ ]:
print(r2_score(y_combi_slice, y_combi_slice_pred))

In [ ]:
y_combi_slice['predictions'] = y_combi_slice_pred.tolist()

In [ ]:
y_combi_slice['dates'] = dates.tolist()

In [ ]:
y_combi_slice.reset_index(inplace=True)
y_combi_slice.plot(x='dates', y=['new_cases_smoothed','predictions'], linestyle="dashed", figsize=(10,10))